In [2]:
def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        print("Run time {:.6f} sec".format(elapsed))
        return result
    return inner


def fibo(n):
    return 1 if n < 3 else fibo(n - 1) + fibo(n - 2)


def fibo_n(n):
    return fibo(n)


fibo_n = timed(fibo_n)
print(fibo_n(10))

Run time 0.000018 sec
55


### В декоратор мы (пока) не можем передавать аргументы. Например, создадим такую функцию

In [6]:
def timed2(fn):
    from functools import wraps
    from time import perf_counter

    def inner(*args, **kwargs):
        total = 0
        for x in range(10):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            elapsed = end - start
            total += elapsed
        avg_total = total / 10
        print("Function {0} needs {1:.6f} sec to run on average".format(
            fn.__name__, avg_total))
        return result
    return inner


def fibo2(n):
    if n < 3:
        return 1
    else:
        return fibo(n - 1) + fibo(n - 2)


@timed2
def fibo2_n(n):
    return fibo2(n)


print(fibo2_n(10))

#в range() мы не можем передать другой аргумент

Function fibo2_n needs 0.000020 sec to run on average
55


### Передадим n таким образом

In [11]:
def timed(fn, reps):
    from functools import wraps
    from time import perf_counter

    def inner(*args, **kwargs):
        total = 0
        for x in range(reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            elapsed = end - start
            total += elapsed
        avg_total = total / reps
        print("Function {0} needs {1:.6f} sec to run on average ({2} reps)".format(
            fn.__name__, avg_total, reps))
        return result
    return inner


def fibo(n):
    if n < 3:
        return 1
    else:
        return fibo(n - 1) + fibo(n - 2)


def fibo_n(n):
    return fibo(n)

fibo_n = timed(fibo_n, 5) # работает с такой записью, но не работает с такой @timed(5)
print(fibo_n(28))

Function fibo_n needs 0.090802 sec to run on average (5 reps)
317811


### Но лучше использовать т.н. decorator factory

In [1]:
# Функция называется dec_factory для примера, чтобы было наглядно, где decorator factory
# Конечно, лучше называть, например, timed (тогда соответственно название внутренней
# функции timed нужно поменять на напр. dec)

def dec_factory(reps):
    def timed(fn):
        from functools import wraps
        from time import perf_counter

        def inner(*args, **kwargs):
            total = 0
            for x in range(reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                elapsed = end - start
                total += elapsed
            avg_total = total / reps
            print("Function {0} needs {1:.6f} sec to run on average ({2} reps)".format(
                fn.__name__, avg_total, reps))
            return result
        return inner
    return timed


def fibo(n):
    if n < 3:
        return 1
    else:
        return fibo(n - 1) + fibo(n - 2)


@dec_factory(5)
def fibo_n(n):
    return fibo(n)

print(fibo_n(28))
# или так, без @dec_factory(5):
# fibo_n = dec_factory(5)(fibo_n)
# print(fibo_n(28))

Function fibo_n needs 0.091640 sec to run on average (5 reps)
317811


<b>Или так без @dec_factory(5)</b>

In [3]:
obj = dec_factory(5)
def fibo_n(n):
    return fibo(n)
fibo_n = obj(fibo_n)
print(fibo_n(28))

Function fibo_n needs 0.090565 sec to run on average (5 reps)
317811
